## Challenge A: Evaulate the PRF of a Aluminate coating

In this exercise you will need to run two FESTIM simulations, one with a 2.5 $\mu \text{m}$ thick barrier 

In [23]:
import festim as F
import h_transport_materials as htm
import numpy as np

# eurofer properties
diffusivity_eurofer = htm.diffusivities.filter(material="eurofer_97").filter(
    author="chen"
)
D_eurofer = diffusivity_eurofer[0]

solubility_eurofer = htm.solubilities.filter(material="eurofer_97").filter(
    author="chen"
)
S_eurofer = solubility_eurofer[0]

# Al2O3 properties
diffusivity_al2o3 = (
    htm.diffusivities.filter(material="alumina")
    .filter(isotope="h")
    .filter(author="serra")
)
D_al2o3 = diffusivity_al2o3[0]
solubility_al2o3 = (
    htm.solubilities.filter(material="alumina")
    .filter(isotope="h")
    .filter(author="serra")
)
S_al2o3 = solubility_al2o3[0]

L_eurofer = 0.02
L_barrier = 2e-6

results_folder = "challenge_A/"


In [24]:
model_with_barrier = F.Simulation(log_level=40)

# define mesh
size = L_eurofer + L_barrier
cells_required = int(size / 1e-07)
vertices = np.linspace(0, size, num=cells_required)

model_with_barrier.mesh = F.MeshFromVertices(vertices=vertices)

# define materials
eurofer = F.Material(
    id=2,
    borders=[0, L_eurofer],
    D_0=D_eurofer.pre_exp.magnitude,
    E_D=D_eurofer.act_energy.magnitude,
    S_0=S_eurofer.pre_exp.magnitude,
    E_S=S_eurofer.act_energy.magnitude,
)
al2o3 = F.Material(
    id=1,
    borders=[L_eurofer, size],
    D_0=D_al2o3.pre_exp.magnitude,
    E_D=D_al2o3.act_energy.magnitude,
    S_0=S_al2o3.pre_exp.magnitude,
    E_S=S_al2o3.act_energy.magnitude,
)
model_with_barrier.materials = F.Materials([eurofer, al2o3])

# define temperature
model_with_barrier.T = F.Temperature(value=600)

# define boundary conditions
model_with_barrier.boundary_conditions = [
    F.DirichletBC(surfaces=1, value=1e18, field="solute"),
    F.DirichletBC(surfaces=2, value=0, field="solute"),
]

# define settings
model_with_barrier.settings = F.Settings(
    absolute_tolerance=1e-10,
    relative_tolerance=1e-10,
    maximum_iterations=30,
    transient=False,
    chemical_pot=True,
)

# define exports
my_derived_quantities = F.DerivedQuantities(
    [        
        F.SurfaceFlux("solute", surface=2)
    ],
    filename=results_folder + "with_barrier.csv", 
)
model_with_barrier.exports = F.Exports([my_derived_quantities])

# run simulation
model_with_barrier.initialise()
model_with_barrier.run()

Defining initial values
Defining variational problem
Defining source terms
Defining boundary conditions
Solving steady state problem...


/home/jdark/anaconda3/envs/festim-env/lib/python3.11/site-packages/festim/exports/derived_quantities/derived_quantities.py:52: DeprecationWarning: The derived_quantities attribute will be deprecated in a future release, please use festim.DerivedQuantities as a list instead
  warnings.warn(


Solved problem in 0.90 s


In [25]:
model_without_barrier = F.Simulation(log_level=40)

# define mesh
size = L_eurofer
cells_required = int(size / 1e-07)
vertices = np.linspace(0, size, num=cells_required)

model_without_barrier.mesh = F.MeshFromVertices(vertices=vertices)

# define materials
eurofer = F.Material(
    id=2,
    borders=[0, L_eurofer],
    D_0=D_eurofer.pre_exp.magnitude,
    E_D=D_eurofer.act_energy.magnitude,
    S_0=S_eurofer.pre_exp.magnitude,
    E_S=S_eurofer.act_energy.magnitude,
)
model_without_barrier.materials = F.Materials([eurofer])

# define temperature
model_without_barrier.T = F.Temperature(value=600)

# define boundary conditions
model_without_barrier.boundary_conditions = [
    F.DirichletBC(surfaces=1, value=1e18, field="solute"),
    F.DirichletBC(surfaces=2, value=0, field="solute"),
]

# define settings
model_without_barrier.settings = F.Settings(
    absolute_tolerance=1e-10,
    relative_tolerance=1e-10,
    maximum_iterations=30,
    transient=False,
    chemical_pot=True,
)

# define exports
my_derived_quantities = F.DerivedQuantities(
    [        
        F.SurfaceFlux("solute", surface=2)
    ],
    filename=results_folder + "without_barrier.csv", 
)
model_without_barrier.exports = F.Exports([my_derived_quantities])

# run simulation
model_without_barrier.initialise()
model_without_barrier.run()

Defining initial values
Defining variational problem
Defining source terms
Defining boundary conditions
Solving steady state problem...


/home/jdark/anaconda3/envs/festim-env/lib/python3.11/site-packages/festim/exports/derived_quantities/derived_quantities.py:52: DeprecationWarning: The derived_quantities attribute will be deprecated in a future release, please use festim.DerivedQuantities as a list instead
  warnings.warn(


Solved problem in 1.00 s


In [26]:
data_with_barrier = np.genfromtxt(
    results_folder + "/with_barrier.csv", delimiter=",", names=True
)
data_without_barrier = np.genfromtxt(
    results_folder + "/without_barrier.csv", delimiter=",", names=True
)

surface_flux_with_barrier = data_with_barrier["Flux_surface_2_solute"] *-1
surface_flux_without_barrier = data_without_barrier["Flux_surface_2_solute"] *-1

PRF = surface_flux_without_barrier / surface_flux_with_barrier

print(f"Surface flux with barrier = {surface_flux_with_barrier:.2e} H/m2/s")
print(f"Surface flux without barrier = {surface_flux_without_barrier:.2e} H/m2/s")

print(f"PRF = {PRF:.0f}")


Surface flux with barrier = 3.06e+08 H/m2/s
Surface flux without barrier = 4.76e+11 H/m2/s
PRF = 1557
